In [1]:
import pandas as pd
import numpy as np

In [3]:
# KOSPI 처리 (12m 로그 수익률 Z-score)
df_kospi_raw = pd.read_csv('data/kospi.csv')
df_kospi_raw['날짜'] = pd.to_datetime(df_kospi_raw['날짜'])
df_kospi_raw = df_kospi_raw.sort_values('날짜')
df_kospi_raw['종가'] = df_kospi_raw['종가'].astype(str).str.replace(',', '').astype(float)
df_kospi = df_kospi_raw.set_index('날짜')[['종가']]
log_ret_kospi = np.log(df_kospi['종가']).diff(12)
df_kospi['kospi'] = (log_ret_kospi - log_ret_kospi.mean()) / log_ret_kospi.std()

In [5]:
# 환율 처리 (12m 로그 수익률 Z-score * -1)
df_fx_raw = pd.read_csv('data/USD_KRW.csv')
df_fx_raw['날짜'] = pd.to_datetime(df_fx_raw['날짜'])
df_fx_raw = df_fx_raw.sort_values('날짜')
df_fx_raw['종가'] = df_fx_raw['종가'].astype(str).str.replace(',', '').astype(float)
df_fx = df_fx_raw.set_index('날짜')[['종가']]
log_ret_fx = np.log(df_fx['종가']).diff(12)
df_fx['exchange'] = -1 * ((log_ret_fx - log_ret_fx.mean()) / log_ret_fx.std())

In [6]:
# 금리/스프레드 처리 (12m 변화량 Z-score * -1)
df_rate_raw = pd.read_csv('data/시장금리.csv')
date_cols = df_rate_raw.columns[4:]
treasury_vals = df_rate_raw[df_rate_raw['계정항목'] == '국고채(3년)'].iloc[0][date_cols].values
company_vals = df_rate_raw[df_rate_raw['계정항목'] == '회사채(3년, AA-)'].iloc[0][date_cols].values
df_rate = pd.DataFrame({
    'Treasury': treasury_vals, 'Company': company_vals
}, index=pd.to_datetime(date_cols, format='%Y/%m'))
df_rate = df_rate.sort_index().astype(float)
df_rate['Spread'] = df_rate['Company'] - df_rate['Treasury']
change_12m = df_rate['Spread'].diff(12)
df_rate['interest'] = -1 * ((change_12m - change_12m.mean()) / change_12m.std())

In [7]:
# 데이터 통합 및 금융지수 계산
combined_fin = df_rate[['interest']].join(df_kospi[['kospi']], how='inner').join(df_fx[['exchange']], how='inner')
combined_fin['fin_index'] = combined_fin.mean(axis=1)

In [9]:
# 형식 맞추기 및 저장
final_fin_output = combined_fin.reset_index().rename(columns={'index': 'date'})
final_fin_output = final_fin_output.dropna()
final_fin_output.to_csv('data/(창민)금융지표계산.csv', index=True, encoding='utf-8-sig')
print("금융 종합지표 파일이 'data/(창민)금융지표계산.csv'로 저장되었습니다.")

금융 종합지표 파일이 'data/(창민)금융지표계산.csv'로 저장되었습니다.
